# Jupyter Notebook Tutorial for Anthropology Course

## Exploring 1000 Genomes Data using Basic Python

### Introduction
We will explore genetic variation, allele frequencies, and the Hardy-Weinberg equilibrium using data from the 1000 Genomes Project.

### Table of Contents
1. [Environment Setup](#Environment-Setup)
2. [Data Import](#Data-Import)
3. [Data Exploration](#Data-Exploration)
4. [Genetic Variation Analysis](#Genetic-Variation-Analysis)
5. [Allele Frequencies](#Allele-Frequencies)
6. [Hardy-Weinberg Equilibrium](#Hardy-Weinberg-Equilibrium)
7. [Conclusion](#Conclusion)

# Environment-Setup

In [ ]:
import importlib.util

# Function to check if a package is installed
def check_install_package(package_name, pip_name=None):
    package_spec = importlib.util.find_spec(package_name)
    if package_spec is None:
        print(f"{package_name} not found. Installing...")
        !pip install {pip_name if pip_name else package_name}
    else:
        print(f"{package_name} is already installed.")

# Check and install scikit-allel if not installed
check_install_package('allel', 'scikit-allel')
check_install_package('zarr', 'zarr')

In [2]:
import os # Imports Python's built-in os library, which allows us to interact with the operating system.
primary_directory = os.getcwd() # get the current working directory and stores it in the variable primary_directory
primary_directory # Displays the value of primary_directory

'/home/lakishadavid/anthropology_methods_course'

If you want to change the value of primary_directory to a different directory, you can manually set it like this:

`primary_directory = "/path/to/your/desired/directory"`

`primary_directory = "/home/username/MyProject"`

In [3]:
# To change primary_directory, replace the path below to the path you want your for your files
primary_directory = '/home/lakishadavid' # for example, this is for my environment
primary_directory

'/home/lakishadavid'

The followng code checks if the directories already exist before attempting to create them.

In [4]:
# Define the other directories
use_directory = os.path.join(primary_directory, "use")
results_directory = os.path.join(primary_directory, "results")
references_directory = os.path.join(primary_directory, "references")
data_directory = os.path.join(primary_directory, "data")

# Create the directories if they do not exist
for directory in [use_directory, results_directory, references_directory, data_directory]:
    if not os.path.exists(directory):
        os.makedirs(directory)

print("Directories have been set up!")

Directories have been set up!


The line `if not os.path.exists(directory):` checks whether a directory with the specified path already exists. If it does not exist, the code inside the if block, `os.makedirs(directory)`, will execute, creating the directory. If the directory already exists, the `os.makedirs(directory)` line will not be executed for that directory, and the code will simply move on to the next directory in the list to check. No new directories will be created if they already exist, and no existing data or files within those directories will be affected.

## Data Import

## Introduction to the IGSR 30x GRCh38 Data Collection
The International Genome Sample Resource (IGSR) provides a data collection for the 30x GRCh38 human genome assembly. This resource is invaluable for researchers and scientists who are working on genomics, as it offers high-quality, publicly available data sets. The 30x coverage ensures a high level of accuracy and reliability for genomic studies.

## How to Download Populations File and Sample File

Normally, you would find options to download various data files, including the populations file and the sample file, on the IGSR data portal. Here's a general outline of how you could download these files and save them to your references_directory:

Navigate to the Data Download Section: Go to the IGSR data portal at https://www.internationalgenome.org/data-portal/data-collection/30x-grch38. Locate the buttons that says "Download the list" in the sample section and the population section.

Download Files: Click on the download links for the populations file and the sample file. Note the directory where these files are saved and the filenames.

Move Files to references_directory: Move these downloaded files to your reference directory. Run the cell below to remind yourself of how to navigate to your reference directory.

In [4]:
# Your reference directory
print(references_directory)

/home/lakishadavid/genetic_genealogy/references


## File Verification

Before proceeding with the data subsetting, let's ensure that the sample and population files you intend to use are available in the `references_directory`.

If you are using VSCode to run this Jupyter Notebook, when you run the next cell, a popup box should appear asking you to enter the filenames for the sample and population files. Please enter the filenames in these boxes to proceed.



In [51]:
sample_file_name = input("Please enter the name of your sample file (e.g., onekg_grch38_samples.tsv) or press ENTER for default: ")
if not sample_file_name:
    sample_file_name = "onekg_grch38_samples.tsv"
print(f"Using sample file: {sample_file_name}")

population_file_name = input("Please enter the name of your population file (e.g., onekg_grch38_populations.tsv) or press ENTER for default: ")
if not population_file_name:
    population_file_name = "onekg_grch38_populations.tsv"
print(f"Using population file: {population_file_name}")

# Check if the files exist in the references_directory
sample_file_path = os.path.join(references_directory, sample_file_name)
population_file_path = os.path.join(references_directory, population_file_name)

# Check if sample file exists
if os.path.exists(sample_file_path):
    print(f"The sample file {sample_file_path} exists.")
else:
    print(f"The sample file {sample_file_path} does not exist.")

# Check if population file exists
if os.path.exists(population_file_path):
    print(f"The population file {population_file_path} exists.")
else:
    print(f"The population file {population_file_path} does not exist.")

Using sample file: onekg_grch38_samples.tsv
Using population file: onekg_grch38_populations.tsv
The sample file /home/lakishadavid/genetic_genealogy/references/onekg_grch38_samples.tsv exists.
The population file /home/lakishadavid/genetic_genealogy/references/onekg_grch38_populations.tsv exists.


## Exploring the Sample and Population Files

Before diving into the analysis, it's a good idea to explore the sample and population files to get a sense of what the data looks like. We'll use Pandas to open these files and display the first few rows.


In [52]:
# Import the Pandas library
import pandas as pd

# Load the sample and population files into Pandas DataFrames
try:
    sample_df = pd.read_csv(sample_file_path, sep='\t')
    population_df = pd.read_csv(population_file_path, sep='\t')
    
    # Display the first few rows of each DataFrame
    print("First few rows of the sample file:")
    display(sample_df.head())
    
    print("First few rows of the population file:")
    display(population_df.head())
    
except FileNotFoundError as e:
    print(f"File not found: {e}")


First few rows of the sample file:


,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,HG00271,male,SAME123417,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG00276,female,SAME123424,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,HG00288,female,SAME1839246,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,HG00290,male,SAME1839057,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
4,HG00303,male,SAME1840115,FIN,Finnish,EUR,European Ancestry,FIN,1000 Genomes on GRCh38


First few rows of the population file:


,Population code,Population elastic ID,Population name,Population description,Population latitude,Population longitude,Superpopulation code,Superpopulation name,Superpopulation display colour,Superpopulation display order,Data collections
0,FIN,FIN,Finnish,Finnish in Finland,60.17000,24.930000,EUR,European Ancestry,#018ead,4,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,CHS,CHS,Southern Han Chinese,Han Chinese South,23.13333,113.266667,EAS,East Asian Ancestry,#778500,3,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,KHV,KHV,Kinh Vietnamese,"Kinh in Ho Chi Minh City, Vietnam",10.78000,106.680000,EAS,East Asian Ancestry,#778500,3,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,BEB,BEB,Bengali,Bengali in Bangladesh,23.70000,90.350000,SAS,South Asian Ancestry,#c44cfd,5,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
4,PUR,PUR,Puerto Rican,Puerto Rican in Puerto Rico,18.40000,-66.100000,AMR,American Ancestry,#710027,2,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


## Data Files Overview

Before diving into the analysis, it's crucial to understand the data files we'll be working with.

---

### Populations File

#### Description
The populations file contains information about the various populations that are part of the genomic study. This file is essential for understanding the diversity of the samples and for performing population-specific analyses.

#### Typical Columns
- **Population ID**: Unique identifier for each population.
- **Population Name**: Name of the population.
- **Region**: Geographical region where the population is located.
- **Number of Samples**: Number of samples collected from this population.
- **Other Metadata**: Additional information such as ethnicity, age range, etc.

#### Use Cases
- Filtering genomic data based on specific populations.
- Performing population-specific genetic variation analyses.
- Understanding the distribution of samples across different populations.

---

### Sample File

#### Description
The sample file contains detailed information about each individual sample that is part of the study. This file is essential for tracking the source of each genomic sequence and for associating it with specific traits or conditions.

#### Typical Columns
- **Sample ID**: Unique identifier for each sample.
- **Population ID**: The population to which the sample belongs.
- **Gender**: Gender of the individual from whom the sample was taken.
- **Age**: Age of the individual.
- **Health Status**: Information about the health condition of the individual, if applicable.
- **Other Metadata**: Additional information such as the date of sample collection, sequencing technology used, etc.

#### Use Cases
- Filtering genomic data based on specific samples or traits.
- Performing individual-level analyses.
- Associating genomic variations with specific traits or conditions.


# Exploratory Data Analysis with Pandas

## Introduction
Before diving into more complex analyses, it's essential to understand the structure and characteristics of your data. Pandas is a powerful Python library that provides fast, flexible, and expressive data structures designed to make working with "relational" or "labeled" data both easy and intuitive. Let's explore some basic Pandas functionalities to better understand our sample and population files.

Remember that we loaded the sample and population files earlier and created Pandas Dataframes called sample_df and population_df.

### Basic Information
You can get a quick overview of the DataFrame using .info().

In [53]:
# Get basic information about the sample DataFrame
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2709 entries, 0 to 2708
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Sample name            2709 non-null   object
 1   Sex                    2709 non-null   object
 2   Biosample ID           2709 non-null   object
 3   Population code        2709 non-null   object
 4   Population name        2709 non-null   object
 5   Superpopulation code   2709 non-null   object
 6   Superpopulation name   2709 non-null   object
 7   Population elastic ID  2709 non-null   object
 8   Data collections       2709 non-null   object
dtypes: object(9)
memory usage: 190.6+ KB


You can preview the dataframe by viewing the first rows using .head() or the last rows using .tail, default 5.

In [54]:
sample_df.head()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,HG00271,male,SAME123417,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG00276,female,SAME123424,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,HG00288,female,SAME1839246,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,HG00290,male,SAME1839057,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
4,HG00303,male,SAME1840115,FIN,Finnish,EUR,European Ancestry,FIN,1000 Genomes on GRCh38


In [55]:
sample_df.head(10)

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,HG00271,male,SAME123417,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG00276,female,SAME123424,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,HG00288,female,SAME1839246,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,HG00290,male,SAME1839057,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
4,HG00303,male,SAME1840115,FIN,Finnish,EUR,European Ancestry,FIN,1000 Genomes on GRCh38
5,HG00308,male,SAME124161,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
6,HG00310,male,SAME124338,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
7,HG00315,female,SAME124335,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
8,HG00327,female,SAME123791,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
9,HG00334,female,SAME123981,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


In [56]:
sample_df.tail()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
2704,NA21143,female,SAME123359,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2705,NA21112,male,SAME124182,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2706,NA21117,male,SAME124185,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2707,NA21124,male,SAME124024,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2708,NA21129,male,SAME124023,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


In [57]:
sample_df.tail(15)

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
2694,NA20897,male,SAME123664,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2695,NA20900,female,SAME122920,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2696,NA20905,male,SAME122915,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2697,NA21092,male,SAME123322,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2698,NA21097,female,SAME123319,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2699,NA21100,male,SAME124440,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2700,NA21105,male,SAME124437,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2701,NA20873,male,SAME122841,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes phase 3 re..."
2702,NA20878,female,SAME123595,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2703,NA20885,male,SAME124525,GIH,Gujarati,SAS,South Asian Ancestry,GIH,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


### Summary Statistics
The .describe() method provides summary statistics of the DataFrame, useful for getting a sense of the distribution of each attribute.

In [58]:
# Get summary statistics for the sample DataFrame
sample_df.describe()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
count,2709,2709,2709,2709,2709,2709,2709,2709,2709
unique,2709,2,2709,27,38,6,10,38,33
top,HG00271,female,SAME123417,GWD,Gambian Mandinka,AFR,African Ancestry,GWD,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
freq,1,1379,1,120,118,719,711,118,1251


### Count Values
To count the number of occurrences of each unique value in a column, you can use .value_counts().

In [60]:
# Count the number of individuals per population
sample_df['Population name'].value_counts()

Population name
Gambian Mandinka            118
Southern Han Chinese        114
Telugu                      112
Toscani                     112
Yoruba                      111
Luhya                       111
Tamil                       111
Dai Chinese                 109
Gujarati                    109
Esan                        109
Han Chinese                 108
Puerto Rican                107
Iberian                     105
Colombian                   105
Japanese                    104
Punjabi                     104
British                     104
CEPH                        103
Finnish                     102
Bengali                     102
Kinh Vietnamese             101
African Caribbean            98
Mende                        96
Peruvian                     91
Mexican Ancestry             70
African Ancestry SW          68
Punjabi,Punjabi               4
Finnish,Finnish               3
British,English               2
Bengali,Bengali               2
Kinh,Kinh Vietnamese    

### Filtering Data
You can filter rows based on certain conditions. For example, let's filter the sample DataFrame to only include individuals from a specific population.

In [62]:
# Filter to include only individuals from the 'YRI' population
yri_population = sample_df[sample_df['Population code'] == 'YRI']
yri_population.head()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
711,NA18489,female,SAME124102,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
713,NA18504,male,SAME125061,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
716,NA18511,female,SAME124885,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
717,NA18516,male,SAME124884,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
740,NA18523,female,SAME124670,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


In [63]:
type(yri_population)

pandas.core.frame.DataFrame

In [64]:
yri_population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 111 entries, 711 to 2651
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Sample name            111 non-null    object
 1   Sex                    111 non-null    object
 2   Biosample ID           111 non-null    object
 3   Population code        111 non-null    object
 4   Population name        111 non-null    object
 5   Superpopulation code   111 non-null    object
 6   Superpopulation name   111 non-null    object
 7   Population elastic ID  111 non-null    object
 8   Data collections       111 non-null    object
dtypes: object(9)
memory usage: 8.7+ KB


In [65]:
yri_population.describe()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
count,111,111,111,111,111,111,111,111,111
unique,111,2,111,1,1,1,1,1,4
top,NA18489,female,SAME124102,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
freq,1,57,1,111,111,111,111,111,89


In [68]:
# Search for individuals with specific attributes
specific_entries = sample_df[(sample_df['Population code'] == 'YRI') & (sample_df['Sex'] == 'female')]
specific_entries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, 711 to 2651
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Sample name            57 non-null     object
 1   Sex                    57 non-null     object
 2   Biosample ID           57 non-null     object
 3   Population code        57 non-null     object
 4   Population name        57 non-null     object
 5   Superpopulation code   57 non-null     object
 6   Superpopulation name   57 non-null     object
 7   Population elastic ID  57 non-null     object
 8   Data collections       57 non-null     object
dtypes: object(9)
memory usage: 4.5+ KB


In [69]:
specific_entries.head()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
711,NA18489,female,SAME124102,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
716,NA18511,female,SAME124885,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
740,NA18523,female,SAME124670,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
752,NA18508,female,SAME124318,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
770,NA18876,female,SAME125103,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


In [70]:
specific_entries.describe()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
count,57,57,57,57,57,57,57,57,57
unique,57,1,57,1,1,1,1,1,4
top,NA18489,female,SAME124102,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
freq,1,57,1,57,57,57,57,57,49


### Multiple Conditions
You can include multiple conditions in your query. For example, let's find all females in either the 'YRI' or 'CEU' populations.

In [71]:
# Search for females in either 'YRI' or 'CEU' populations
multiple_conditions = sample_df[(sample_df['Population code'].isin(['YRI', 'CEU'])) & (sample_df['Sex'] == 'female')]
multiple_conditions.describe()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
count,110,110,110,110,110,110,110,110,110
unique,110,1,110,2,2,2,2,2,7
top,NA12828,female,SAME125076,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
freq,1,110,1,57,57,57,57,57,95


In [72]:
target_population = 'YRI'
target_sex = 'female'

# Search using variables
variable_filter = sample_df[(sample_df['Population code'] == target_population) & (sample_df['Sex'] == target_sex)]
variable_filter.describe()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
count,57,57,57,57,57,57,57,57,57
unique,57,1,57,1,1,1,1,1,4
top,NA18489,female,SAME124102,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
freq,1,57,1,57,57,57,57,57,49


### Searching for Entries in a List
If you have a list of values to search for, you can use the isin() method within .query().

In [73]:
# List of target populations
target_populations = ['YRI', 'CEU']

# Search for individuals in target populations
list_filter = sample_df[sample_df['Population code'].isin(target_populations)]
list_filter.describe()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
count,214,214,214,214,214,214,214,214,214
unique,214,2,214,2,2,2,2,2,7
top,NA12828,female,SAME125076,YRI,Yoruba,AFR,African Ancestry,YRI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
freq,1,110,1,111,111,111,111,111,179


### Using String Methods
You can also use string methods to search for specific patterns in string columns.

In [77]:
# Search for individuals whose sample IDs start with 'NA'
string_filter = sample_df[sample_df['Sample name'].str.startswith('NA')]
print(len(string_filter))
string_filter.head()

899


,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
698,NA12828,female,SAME125076,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
700,NA12830,female,SAME125289,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
702,NA12842,male,SAME123542,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
704,NA12873,female,SAME123388,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
706,NA12878,female,SAME123392,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


## Exploratory Data Analysis

Before we move on, let's specifically look at:

1. The total number of samples in the dataset.
2. The distribution of samples by sex.
3. The distribution of samples by population.
4. The distribution of samples by superpopulation.

In [78]:
# Check if the sample DataFrame is loaded
if 'sample_df' in locals():

    # Total number of samples
    total_samples = len(sample_df)
    print(f"Total number of samples: {total_samples}")

    # Distribution by Sex
    print("\nDistribution of samples by Sex:")
    display(sample_df['Sex'].value_counts())

    # Distribution by Superpopulation
    print("\nDistribution of samples by Superpopulation:")
    display(sample_df['Superpopulation name'].value_counts())

else:
    print("The sample DataFrame is not loaded. Please make sure to load the sample file.")

Total number of samples: 2709

Distribution of samples by Sex:


Sex
female    1379
male      1330
Name: count, dtype: int64


Distribution of samples by Superpopulation:


Superpopulation name
African Ancestry                          711
South Asian Ancestry                      538
East Asian Ancestry                       536
European Ancestry                         526
American Ancestry                         373
African Ancestry,Africa (SGDP)              8
European Ancestry,West Eurasia (SGDP)       7
South Asia (SGDP),South Asian Ancestry      6
East Asia (SGDP),East Asian Ancestry        3
European Ancestry,African Ancestry          1
Name: count, dtype: int64

## Downloading 1000 Genomes Data

Up to this point, we have explored the sample and population files to understand the structure of our data. However, we have not yet downloaded the actual genetic data from the 1000 Genomes Project. 

In the next step, we will download the VCF (Variant Call Format) files for each chromosome. These files contain the genetic variants for our samples and are essential for our subsequent analyses.

Note: The script will also check if the directory for storing the 1000 Genomes reference panel exists. If not, it will create one for you.

Let's proceed to download the data.

In [14]:
%%bash -s "$primary_directory" "$references_directory" "$results_directory"

# Receive directory variables from Python
primary_dir=$1
ref_dir=$2
results_dir=$3

# Define the directory for the 1000 Genomes reference panel
onekg_reference_panel_dir="${ref_dir}/onekg_reference_panel"

# Check if the onekg_reference_panel directory exists; if not, create it
if [ ! -d "${onekg_reference_panel_dir}" ]; then
    echo "Creating onekg_reference_panel directory..."
    mkdir -p "${onekg_reference_panel_dir}"
fi

echo "These files are very large so expect to let this run for several hours."
echo

# Loop through each chromosome to download data
for chromosome in {1..22}
do
    # Download the VCF file for each chromosome from the 1000 Genomes FTP site
    echo "Downloading chromosome ${chromosome}..."
    wget --quiet ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/release/20190312_biallelic_SNV_and_INDEL/ALL.chr${chromosome}.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz \
        -P ${onekg_reference_panel_dir}
done

These files are very large so expect to let this run about a day.



## Subsetting to 500 Individuals

### Why Is It Necessary?

As we dive deeper into genetic data analysis, it's important to manage computational resources effectively. Whole-genome VCF files from projects like the 1000 Genomes can be extremely large, often containing data for thousands of individuals across millions of genetic variants. Loading such extensive data into memory can be computationally intensive and may even cause the kernel to crash, as we've experienced.

### Advantages of Subsetting

1. **Reduced Computational Load**: By focusing on a subset of 500 individuals, we significantly reduce the computational resources needed for the analysis.

2. **Faster Execution**: Smaller datasets mean that code will execute more quickly, allowing us to focus on the analysis rather than waiting for code to run.

3. **Feasibility**: Not all personal computers will have the resources to handle large genomic datasets. Subsetting makes the tutorial more accessible to individuals with less powerful machines.

4. **Focused Analysis**: With fewer individuals, it's easier to explore the data in depth, which is particularly useful for educational purposes.

### Random Selection

The 500 individuals are randomly selected to ensure that the subset is representative of the larger dataset. This allows us to make more general conclusions from our analyses.

In the next cell, we will use `bcftools` to subset our VCF file to include only these 500 randomly selected individuals. This will make subsequent analyses more manageable and less resource-intensive.

In [1]:
# Ubuntu
import os

# Check if bcftools is installed
bcftools_installed = os.system("which bcftools")

# Install bcftools if not installed
if bcftools_installed != 0:
    print("bcftools is not installed. Get in instructions on how to install.")
    # os.system("sudo apt-get update")
    # os.system("sudo apt-get install -y bcftools")
else:
    print("bcftools is already installed.")

/bin/bcftools
bcftools is already installed.


In [4]:
%%bash -s "$references_directory"

# Receive directory variables from Python
ref_dir=$1

# Define the directory for the 1000 Genomes reference panel
onekg_reference_panel_dir="${ref_dir}/onekg_reference_panel"

# Extract sample names from the VCF header
bcftools query -l ${onekg_reference_panel_dir}/ALL.chr20.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz > ${onekg_reference_panel_dir}/all_sample_ids.txt

# Randomly select 500 sample names
shuf -n 500 ${onekg_reference_panel_dir}/all_sample_ids.txt > ${onekg_reference_panel_dir}/random_500_sample_ids.txt

# Subset the VCF file based on the random 500 sample IDs
bcftools view -S ${onekg_reference_panel_dir}/random_500_sample_ids.txt -o ${onekg_reference_panel_dir}/subset_500_chr20.vcf.gz -Oz ${onekg_reference_panel_dir}/ALL.chr20.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz


## Exploring the VCF Files

Now that we have downloaded the VCF files and selected a subset of 500 indiviuals, it's time to explore the VCF files. VCF (Variant Call Format) files contain information about genetic variants found in the samples.

In this section, we will:
1. Load a VCF file for a specific chromosome.
2. Explore the structure of the VCF file.
3. Examine some basic statistics.

Let's get started!

### Installing Required Packages

In data analysis and scientific computing, we often rely on specialized packages to perform specific tasks. These packages are collections of functions and methods that allow us to perform operations without having to write code from scratch.

Before we can use these packages, we need to install them. This is usually a one-time operation. Below, we will install the `scikit-allel` package, which provides tools for bioinformatics and genomics, particularly in the domain of high-throughput sequencing.

Let's proceed to install the package.

### Loading a VCF File

We will start by loading a VCF file for a specific chromosome. For demonstration purposes, let's focus on chromosome 1.

In [4]:
import allel  # Importing the scikit-allel package

# Define the path to the subsetted VCF file
vcf_path_chromosome_20_subset = os.path.join(references_directory, "onekg_reference_panel", "subset_500_chr20.vcf.gz")

# Check if the subsetted VCF file exists
if os.path.exists(vcf_path_chromosome_20_subset):
    # Load the subsetted VCF file
    callset = allel.read_vcf(vcf_path_chromosome_20_subset)
    
    # Display the keys to understand the structure
    print("Keys in the VCF file:", list(callset.keys()))
else:
    print(f"The subsetted VCF file does not exist at {vcf_path_chromosome_20_subset}. Please make sure the file is in the correct location.")

# NOTE: add instructions for if it crash here.

print("Notice the output are the keys in the VCF file. We will use the second key, 'calldata/GT', 'variants/ALT', and 'variants/REF'")


Keys in the VCF file: ['samples', 'calldata/GT', 'variants/ALT', 'variants/CHROM', 'variants/FILTER_PASS', 'variants/ID', 'variants/POS', 'variants/QUAL', 'variants/REF']
Notice the output are the keys in the VCF file. We will use the second key, 'calldata/GT', 'variants/ALT', and 'variants/REF'


## Understanding Allele Frequencies and SNPs

Before we proceed to subset our data, it's essential to understand some key genetic concepts: Allele Frequencies and Single Nucleotide Polymorphisms (SNPs).

### Allele Frequencies

Allele frequencies describe how often a particular variant (allele) of a gene appears within a given population. It is usually expressed as a proportion or percentage. Understanding allele frequencies is crucial for studying genetic diversity and for identifying alleles that may be associated with specific traits or diseases.

### Single Nucleotide Polymorphisms (SNPs)

SNPs are variations at a single position in a DNA sequence among individuals. They are the most common type of genetic variation and serve as markers for locating genes associated with diseases or specific traits.

### What Makes a SNP a SNP?
A Single Nucleotide Polymorphism occurs when a single nucleotide (A, T, C, or G) in the genome sequence is altered. For a variation to be considered a SNP, it must occur in at least 1% of the population. This distinguishes SNPs from random mutations, which are rare and may occur in any individual. SNPs can be synonymous (do not change the protein sequence) or non-synonymous (change the protein sequence), and they can occur in coding or non-coding regions of the genome.

### Assumptions in SNP Analysis
In SNP analysis, it is generally assumed that the nucleotide sequences between the SNPs are conserved, or identical, across the individuals being studied. This assumption allows us to focus on the SNPs as markers of genetic variation without having to analyze the entire genome.

This is a reasonable assumption because:

Most of the human genome is highly conserved.
SNPs are the most common form of genetic variation, making them effective markers for genetic diversity.
By focusing on SNPs, we can efficiently study genetic variation and its implications for traits, diseases, and population history.

### Relatedness and Shared SNPs

#### What Does It Mean If Two Individuals Share the Same SNP?

Sharing a single SNP between two individuals doesn't necessarily imply a close genetic relationship, as SNPs can be quite common in populations. However, sharing the same SNPs over a specific length of DNA sequence can be a strong indicator of relatedness. **The length of the DNA sequence where the SNPs are shared is crucial.** A longer stretch of shared SNPs increases the likelihood that the two individuals are closely related. This is often measured in centimorgans (cM), a unit that describes the genetic distance between positions on a chromosome.

#### Implications for Relatedness

1. **Close Relatives**: Close relatives like siblings or parent-child pairs will share long stretches of SNPs.
  
2. **Distant Relatives**: More distant relatives like cousins may share shorter stretches but still longer than what would be expected by random chance.

3. **Very Distantly Related Individuals**: In individuals who share a distant common ancestry, any shared SNPs are likely to be short, often less than 1 centimorgan (cM). These short stretches are scattered randomly across the genome. While they are not indicative of a close or recent familial relationship, they do reflect shared ancestors from a distant past.

By analyzing the length and distribution of shared SNPs, researchers can infer the degree of relatedness between individuals, which is valuable in studies including Migration Patterns, Adaptation Studies, Archaeogenetics, Forensic Anthropology, and Evolutionary Anthropology.

### Identifying SNPs

1. **Total SNPs**: The total number of SNPs identified in the dataset gives us an overview of the genetic variation present.
  
2. **SNPs by Superpopulation**: Breaking down the SNPs by superpopulation allows us to understand how genetic variations are distributed among different groups. This is crucial for studies that aim to understand the genetic basis of population-specific traits or susceptibilities.

In [5]:
import numpy as np

gt = allel.GenotypeArray(callset['calldata/GT'])

# Get the reference and alternate alleles from the VCF file.
ref_alleles = callset['variants/REF']
alt_alleles = callset['variants/ALT'][:, 0] #  to extract the first alternate allele for each variant. 
# This slice operation is taking all rows (:) and the first column (0) of the alt_alleles array

# Create a boolean mask for biallelic variants
is_biallelic = np.array([len(set(allele) - {'', '.'}) == 1 for allele in alt_alleles])

# Create a boolean mask for valid SNPs
valid_bases = {'A', 'T', 'C', 'G'}
is_valid_snp = np.array([ref in valid_bases and alt in valid_bases for ref, alt in zip(ref_alleles, alt_alleles)])

# Combine the two masks
final_mask = is_biallelic & is_valid_snp

# Apply the mask to the genotype array
gt_filtered = gt.compress(final_mask, axis=0)

print(f"Total number of samples: {gt.shape[1]}")
print(f"Total number of variants: {gt.shape[0]:,}")
print(f"Total number of biallelic variants: {gt_filtered.shape[0]:,}")

Total number of samples: 500
Total number of variants: 1,817,492
Total number of biallelic variants: 1,706,442


### Understanding the Shape of the Genotype Array

In the code, `gt` is a `GenotypeArray` object that contains the genotype data. The `shape` attribute of this array provides us with the dimensions of the array. Specifically, `gt.shape` returns a tuple with the following information:

- `gt.shape[0]`: This represents the total number of variants (or genetic markers like SNPs) in the dataset.
- `gt.shape[1]`: This represents the total number of samples (or individuals) in the dataset.

By printing these values, we get a quick overview of the dataset's size, both in terms of the number of variants it contains and the number of samples it pertains to. This is valuable for understanding the scope of the data we are working with.

### What is an Array?

An array is a data structure that stores a collection of items, typically of the same type, at contiguous memory locations. The elements can be accessed randomly by indexing into the array. In programming languages like Python, arrays are often used for data manipulation and storage.

### Understanding `gt.shape` in scikit-allel's GenotypeArray

The `gt.shape` attribute in the context of a `GenotypeArray` from scikit-allel returns a tuple with three values, each representing a specific dimension of the genetic data:

1. **`n_variants`**: The first value `gt.shape[0]` represents the number of genetic variants in the dataset, such as SNPs and indels.
  
2. **`n_samples`**: The second value `gt.shape[1]` represents the number of individual samples in the dataset.

3. **`ploidy`**: The third value `gt.shape[2]` represents the ploidy of the organism, which is the number of sets of chromosomes in a cell. In simpler terms, it's the number of alleles for each variant.

#### Why Use Arrays?

Arrays are particularly useful when you need to store and manipulate large amounts of data efficiently. They are often used in mathematical computations, data analysis, and algorithmic implementations because of their efficiency in random access and data manipulation.

In the context of genetics and bioinformatics, arrays can be used to store large datasets like genotype information, sequence data, and much more, allowing for efficient computations and analyses.

### Calculate allele counts

We use our gt variable that we created to calculate the allele counts using the count_alleles() method. 
This gives us the number of occurrences of each allele for each variant.


In [6]:
ac = gt_filtered.count_alleles()
print(ac[:5]) # print the first 5 variants

max_allele_count = ac.max(axis=1)
total_allele_count = ac.sum(axis=1)

does_vary = (total_allele_count - max_allele_count) > 0
# does_vary = (ac.sum(axis=1) - ac.max(axis=1)) > 0
print(f"In many cases, all 500 individuals (1,000 haplotpes) have the same variant. However, there is some variability.")
print(f"Total number of genetic variants that vary: {np.count_nonzero(does_vary):,}")

1000    0
1000    0
1000    0
1000    0
 999    1

In many cases, all 500 individuals (1,000 haplotpes) have the same variant. However, there is some variability.
Total number of genetic variants that vary: 878,069


**What is ac?**

`ac` is an Allele Counts Array, where each row represents a variant (like a SNP or an indel), and each column represents an allele. 

For bi-allelic SNPs, you'll typically have two columns: one for each allele (e.g., A and T).

Here's a simplified example of what ac might look like for 3 variants:

```
20  30  # Variant 1: 20 counts of allele 1, 30 counts of allele 2
40  10  # Variant 2: 40 counts of allele 1, 10 counts of allele 2
25  25  # Variant 3: 25 counts of allele 1, 25 counts of allele 2
```
**What does axis=1 mean?**

In NumPy (and many other Python libraries), an array can have multiple dimensions. The axis parameter specifies which dimension you want to perform the operation along.

`axis=0` means that the operation will be performed vertically (down the rows for each column).
`axis=1` means that the operation will be performed horizontally (across the columns for each row).

For our example ac array:

The sum for Variant 1 would be `20 + 30 = 50`

The sum for Variant 2 would be `40 + 10 = 50`

The sum for Variant 3 would be `25 + 25 = 50`

So, ac.sum(axis=1) would return `[50, 50, 50]` in this example.

**Why is this useful?**

Summing the allele counts for each variant gives you the total number of alleles observed for that variant. This is useful for various types of genetic analyses, including identifying SNPs, calculating allele frequencies, and more.

`ac.max(axis=1)`

This finds the maximum allele count for each variant. Essentially, it identifies the most common allele for each variant.

`(ac.sum(axis=1) - ac.max(axis=1)) > 0`

This expression calculates the difference between the total number of alleles and the count of the most common allele for each variant. The idea is to find out how many of the "other" alleles are present.

For example, if a variant has allele counts `[20, 30]`, the sum would be `50`, and the max would be `30`. The difference `50 - 30` would be `20`, representing the count of the less common allele.

If the value of `ac.max(axis=1)` is the same as `ac.sum(axis=1)`, it means that all observed alleles for that particular variant are the same. In other words, there is only one type of allele present for that variant.

For example, let's say a variant has allele counts `[50, 0]`. The sum would be `50`, and the max would also be `50`. The difference `50 - 50` would be `0`, and `(ac.sum(axis=1) - ac.max(axis=1)) > 0` would evaluate to `False` for this variant, indicating that it is not a SNP.

So, when the sum and the max are the same, the variant is not considered a Single Nucleotide Polymorphism (SNP) because there is no variation; all observed alleles are the same.

If the count of the less common allele is greater than zero, that means there is more than one type of allele present for that variant, making it a Single Nucleotide Polymorphism (SNP).

`np.count_nonzero(is_snp)`
Finally, this counts the number of `True` values in the `is_snp` array, giving you the total number of SNPs identified.

Putting it all together
The line `is_snp = (ac.sum(axis=1) - ac.max(axis=1)) > 0` is a way to identify SNPs by checking if there is more than one type of allele present for each variant. Then, np.count_nonzero(is_snp) counts how many SNPs have been identified.

## Minor Allele Frequency
What is Minor Allele Frequency?

In a given population, for a particular genetic variant (or SNP), the allele that occurs less frequently is termed the "minor allele." The frequency of this minor allele in the population is known as the Minor Allele Frequency (MAF). It is calculated as the count of the minor allele divided by the total number of alleles examined.

**Why is MAF Important?**

Statistical Power: Variants with extremely low MAF are often excluded because they may lack the statistical power to detect association with a trait or disease.

Quality Control: Filtering by MAF is a common quality control step to remove potential errors in variant calling.

Biological Relevance: Variants with higher MAF are more likely to be biologically relevant and less likely to be random mutations.

Calculating MAF with scikit-allel
In scikit-allel, you can calculate the MAF as follows:

In [7]:
print(f"Of the {gt_filtered.shape[0]:,} variants in chromosome 20, we determine that {np.count_nonzero(does_vary):,} vary, but are not necessarly SNPs.")

Of the 1,706,442 variants in chromosome 20, we determine that 878,069 vary, but are not necessarly SNPs.


In [7]:
# Calculate Minor Allele Frequency (MAF)

# max_allele_count = ac.max(axis=1)
# total_allele_count = ac.sum(axis=1)

minor_allele_count = total_allele_count - max_allele_count

maf = (total_allele_count - max_allele_count) / total_allele_count


# Here, gt.compress(maf > 0.05, axis=0) will keep only the rows (variants) in the genotype array where the MAF is greater than 5%.
# A set of SNPs with a MAF greater than 5%
gt_maf_filtered_5 = gt_filtered.compress(maf > 0.05, axis=0)
print(f"Total number of common SNPs accross the whole dataset: {gt_maf_filtered_5.shape[0]:,}")

Total number of SNPs accross the whole dataset: 280,203
Total number of common SNPs accross the whole dataset: 158,391


### Summary
Filtering by MAF is a crucial step in many genetic analyses. It helps in focusing on variants that are likely to be meaningful, thereby increasing the robustness and reliability of your results.